# Project: Marvel Cinematic Universe's Acquintance network


This project is about the MCU and its acquintance network created by different actors / actresses. This means that we have to find all connections, weight the network accordingly and give different measured parameters of it.

## Data Cleaning

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

import sys
import os
import glob
import pathlib

import re
import uuid

In [2]:
mcu_orig = pd.read_csv("data/characters.csv")
display(mcu_orig.head())
display(mcu_orig.info())

mcu_sorted0 = mcu_orig.sort_values(by = 'character')

,character,Ant-Man,Avengers: Age of Ultron,Avengers: Endgame,Avengers: Infinity War,Black Panther,Captain America: Civil War,Captain America: The First Avenger,Captain America: The Winter Soldier,Captain Marvel,...,Spider-Man: Homecoming,The Avengers,Thor,Thor: Ragnarok,Thor: The Dark World,total lines,total words,movie appearances,avg words per line,avg lines per movie
0,TONY STARK,0,173,137,119,0,165,0,0,0,...,74,152,0,0,0,1788,22590,9,12,198
1,STEVE ROGERS,3,133,119,25,0,154,186,218,3,...,7,121,0,0,0,977,8653,10,8,97
2,THOR,0,55,70,74,0,0,0,0,0,...,0,47,166,289,133,863,9487,7,10,123
3,PETER PARKER,0,0,7,39,0,73,0,0,0,...,402,0,0,0,0,551,5517,4,10,137
4,NATASHA ROMANOFF,1,79,69,15,0,64,0,112,3,...,0,97,0,1,0,498,4689,10,9,49


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   character                            652 non-null    object
 1   Ant-Man                              652 non-null    int64 
 2   Avengers: Age of Ultron              652 non-null    int64 
 3   Avengers: Endgame                    652 non-null    int64 
 4   Avengers: Infinity War               652 non-null    int64 
 5   Black Panther                        652 non-null    int64 
 6   Captain America: Civil War           652 non-null    int64 
 7   Captain America: The First Avenger   652 non-null    int64 
 8   Captain America: The Winter Soldier  652 non-null    int64 
 9   Captain Marvel                       652 non-null    int64 
 10  Guardians of the Galaxy Vol. 2       652 non-null    int64 
 11  Iron Man                             652 non-

None

In [116]:
display(mcu_sorted0['character'].values)

array(['25TH FLOOR STRIKE AGENT', '4F DOCTOR', '911 OPERATOR',
       'A1 CLINT BARTON', 'A1 HULK', 'A1 NATASHA', 'A1 STEVE', 'A1 THOR',
       'A1 TONY', 'AARON DAVIS', 'ABRAHAM BROWN', 'ABU', 'ACCUSER',
       'ACTOR LOKI', 'ACTOR ODIN', 'ACTOR SIF', 'ACTOR THOR', 'ADMIRAL',
       'ADMISSIONS NURSE', 'ADRIAN TOOMES', 'AGENT', 'AGENT CALE',
       'AGENT DELANCEY', 'AGENT GARRETT', 'AGENT JACKSON', 'AKIHIKO',
       'ALDRICH KILLIAN', 'ALETA', 'ALEXANDER PIERCE', 'ALGRIM',
       'ALPHA GUARD', 'ANCIENT ONE', 'ANNE MARIE HOAG', 'ANNIE',
       'ANNOUNCEMENT', 'ANNOUNCER', 'ANTIQUE STORE OWNER', 'ANTON VANKO',
       'APPLE EMPLOYEE', 'ARMY HECKLER#1', 'ARMY HECKLER#2',
       'ARMY HECKLER#3', 'ARMY HECKLER#4', 'ASGARDIAN CITIZENS',
       'ASGARDIAN EINHERJAR', 'ASGARDIAN MAN', 'ASGARDIAN NURSE',
       'ASGARDIAN PA', 'ASGARDIAN WOMAN', 'ATT-LASS', 'ATTACHE',
       'ATTENDANT', 'AVENGERS JIMMY', 'AVENGERS KID', 'AYESHA', 'AYO',
       'BAD NEBULA', 'BAILIFF', 'BALLET INSTRUCTOR', 

**Problem:**

It seems like multiple rows are in here that has multiple characters in it. This may mean that these chaacters are actually talking together. These have to be separated into rows and it should be compared if there are multiple rows that actually match. **THIS MEANS I HAVE TO READ THROUGH IT MANUALLY**

In [132]:
mcu_sorted0 = mcu_orig.sort_values(by = 'character')

In [133]:
char_list = [",", "/"]  #chars to match
multiple_ppl = []
for i in mcu_sorted0['character'].values:
    multiple_ppl.append(any([characters in char_list for characters in i]))

print(len(mcu_sorted0['character'].values[multiple_ppl]))
print(mcu_sorted0['character'].values[multiple_ppl])

10
['ELDERS, WARRIORS, DORA' 'JASON/BETTY' 'JOSEF / SUPER SOLDIER #1'
 'LIZ/NED' 'LOKI/STEVE' 'SHURI/NAKIA' 'THOR / BANNER' 'THOR/SELVIG'
 'VERS, BRON-CHAR, KORATH, MINN-ERVA AND ATT-LASS' 'VOLSTAGG / HOGUN']


This means there are 10 rows depicting multiple people as on character, because that's how the transcript has it. It must be expected that these people may not be here individually.

**It doesn't matter how much I want to avoid doing this manually, I have to do it manually** (technically a robust regex could handle it)

In [134]:
#this means that i have to add them manually
any(mcu_sorted0['character'].values == 'ELDERS')

False

In [135]:
my_list = [['ELDERS','WARRIORS','DORA'],
           ['JASON','BETTY'],
           ['JOSEF', 'SUPER SOLDIER #1'],
           ['LIZ', 'NED'],
           ['LOKI','STEVE'],
           ['SHURI', 'NAKIA'],
           ['THOR', 'BANNER'],
           ['THOR','SELVIG'],
           ['VERS', 'BRON-CHAR', 'KORATH', 'MINN-ERVA', 'ATT-LASS'],
           ['VOLSTAGG', 'HOGUN']]

In [136]:
mcu_sorted1 = mcu_sorted0.copy()
"""
for i in range(len(my_list)):
    #get loc for values
    loc = np.argmax(mcu_sorted0['character'].values == mcu_sorted0['character'][multiple_ppl].values[i])
    values0 = mcu_sorted0.iloc[loc].values
    print("\n",loc,values)
    for j in my_list[i]:
        #check if its in
        _isin = any(mcu_sorted0['character'].values == j)

        if not _isin:
            #if not in add
            temp =np.append(np.array(j),(values0[1:]))
            mcu_sorted1.loc[len(mcu_sorted1.index)] = temp
            print('Adding ',temp)
            
        else:
            #if in, modify by getting old values and adding them.
            loc2 = np.argmax(mcu_sorted0['character'].values == j)
            val2 = np.array(mcu_sorted0.iloc[loc2].values[1:])+ np.array(values0[1:])
            inserting =  np.append(np.array(j),(val2))
            print('Modifying ',inserting)
            mcu_sorted1.iloc[loc2] = inserting         
"""            
#go through the values and drop the rows that has multiple people in it
new_mul_ppl = []
for i in mcu_sorted1['character'].values:
    new_mul_ppl.append(any([characters in char_list for characters in i]))

mcu_sorted1 = mcu_sorted1[~np.array(new_mul_ppl)]

#drop columns tha are not needed

mcu_sorted1 = mcu_sorted1.drop(columns = mcu_sorted0.columns[-5:])

In [137]:
mcu_sorted1

,character,Ant-Man,Avengers: Age of Ultron,Avengers: Endgame,Avengers: Infinity War,Black Panther,Captain America: Civil War,Captain America: The First Avenger,Captain America: The Winter Soldier,Captain Marvel,Guardians of the Galaxy Vol. 2,Iron Man,Iron Man 2,Iron Man 3,Spider-Man: Homecoming,The Avengers,Thor,Thor: Ragnarok,Thor: The Dark World
500,25TH FLOOR STRIKE AGENT,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
192,4F DOCTOR,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0
556,911 OPERATOR,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
477,A1 CLINT BARTON,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
417,A1 HULK,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,YOUNG TONY STARK,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0
230,YOUNG ZURI,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0
338,ZEMO'S WIFE,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
531,ZRINKA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


I am idiot, as I forgot that hte people that talk together normally have their own individual lines, thus i don't need to play with these... just remove them.

But there are with multiple names, so these have to be combined

In [138]:
display(mcu_sorted1.iloc[np.argmax(mcu_sorted1['character'] == 'A1 THOR')])
display(mcu_sorted1.iloc[np.argmax(mcu_sorted1['character'] == 'THOR')])
display(mcu_sorted1.iloc[np.argmax(mcu_sorted1['character'] == 'ACTOR THOR')])

character                              A1 THOR
Ant-Man                                      0
Avengers: Age of Ultron                      0
Avengers: Endgame                            5
Avengers: Infinity War                       0
Black Panther                                0
Captain America: Civil War                   0
Captain America: The First Avenger           0
Captain America: The Winter Soldier          0
Captain Marvel                               0
Guardians of the Galaxy Vol. 2               0
Iron Man                                     0
Iron Man 2                                   0
Iron Man 3                                   0
Spider-Man: Homecoming                       0
The Avengers                                 0
Thor                                         0
Thor: Ragnarok                               0
Thor: The Dark World                         0
Name: 221, dtype: object

character                              THOR
Ant-Man                                   0
Avengers: Age of Ultron                  55
Avengers: Endgame                        70
Avengers: Infinity War                   74
Black Panther                             0
Captain America: Civil War                0
Captain America: The First Avenger        0
Captain America: The Winter Soldier       0
Captain Marvel                            0
Guardians of the Galaxy Vol. 2            0
Iron Man                                  0
Iron Man 2                                0
Iron Man 3                                0
Spider-Man: Homecoming                    0
The Avengers                             47
Thor                                    166
Thor: Ragnarok                          289
Thor: The Dark World                    133
Name: 2, dtype: object

character                              ACTOR THOR
Ant-Man                                         0
Avengers: Age of Ultron                         0
Avengers: Endgame                               0
Avengers: Infinity War                          0
Black Panther                                   0
Captain America: Civil War                      0
Captain America: The First Avenger              0
Captain America: The Winter Soldier             0
Captain Marvel                                  0
Guardians of the Galaxy Vol. 2                  0
Iron Man                                        0
Iron Man 2                                      0
Iron Man 3                                      0
Spider-Man: Homecoming                          0
The Avengers                                    0
Thor                                            0
Thor: Ragnarok                                 12
Thor: The Dark World                            0
Name: 143, dtype: object

In [140]:
collect_dropping = ['A1 CLINT BARTON',
                    'A1 HULK',
                    'A1 NATASHA',
                    'A1 STEVE',
                    'A1 THOR',
                    'A1 TONY',             #A1 is some alternative version of the avengers
                    'BOTH',                 #no clue who is this
                   ]
for i in collect_dropping:
    mcu_sorted1 = mcu_sorted1[~(mcu_sorted1['character'] == i)]
    
display(mcu_sorted1['character'].values)

array(['25TH FLOOR STRIKE AGENT', '4F DOCTOR', '911 OPERATOR',
       'AARON DAVIS', 'ABRAHAM BROWN', 'ABU', 'ACCUSER', 'ACTOR LOKI',
       'ACTOR ODIN', 'ACTOR SIF', 'ACTOR THOR', 'ADMIRAL',
       'ADMISSIONS NURSE', 'ADRIAN TOOMES', 'AGENT', 'AGENT CALE',
       'AGENT DELANCEY', 'AGENT GARRETT', 'AGENT JACKSON', 'AKIHIKO',
       'ALDRICH KILLIAN', 'ALETA', 'ALEXANDER PIERCE', 'ALGRIM',
       'ALPHA GUARD', 'ANCIENT ONE', 'ANNE MARIE HOAG', 'ANNIE',
       'ANNOUNCEMENT', 'ANNOUNCER', 'ANTIQUE STORE OWNER', 'ANTON VANKO',
       'APPLE EMPLOYEE', 'ARMY HECKLER#1', 'ARMY HECKLER#2',
       'ARMY HECKLER#3', 'ARMY HECKLER#4', 'ASGARDIAN CITIZENS',
       'ASGARDIAN EINHERJAR', 'ASGARDIAN MAN', 'ASGARDIAN NURSE',
       'ASGARDIAN PA', 'ASGARDIAN WOMAN', 'ATT-LASS', 'ATTACHE',
       'ATTENDANT', 'AVENGERS JIMMY', 'AVENGERS KID', 'AYESHA', 'AYO',
       'BAD NEBULA', 'BAILIFF', 'BALLET INSTRUCTOR', 'BARKEEPER',
       'BARON WOLFGANG VON STRUCKER', 'BARTENDER', 'BBC ANCHOR',
       